In [ ]:
#Do not run this block

/* Q1: Some of the facilities charge a fee to members, but some do not.
Write a SQL query to produce a list of the names of the facilities that do. */

SELECT name FROM `Facilities` WHERE `membercost` > 0

/* Q2: How many facilities do not charge a fee to members? */


SELECT COUNT(name) FROM `Facilities` WHERE `membercost` > 0

/* Q3: Write an SQL query to show a list of facilities that charge a fee to members,
where the fee is less than 20% of the facility's monthly maintenance cost.
Return the facid, facility name, member cost, and monthly maintenance of the
facilities in question. */

SELECT `name` , `facid` , `membercost` , `monthlymaintenance`
FROM `Facilities`
WHERE `membercost` >0
AND ( .2 * `monthlymaintenance` > `guestcost` )

/* Q4: Write an SQL query to retrieve the details of facilities with ID 1 and 5.
Try writing the query without using the OR operator. */

SELECT * FROM `Facilities` WHERE facid = 1  OR facid = 5


/* Q5: Produce a list of facilities, with each labelled as
'cheap' or 'expensive', depending on if their monthly maintenance cost is
more than $100. Return the name and monthly maintenance of the facilities
in question. */

SELECT name, monthlymaintenance,
CASE WHEN (monthlymaintenance > 100) THEN'expensive'
ELSE 'cheap'
END AS 'cost'
FROM `Facilities` 


/* Q6: You'd like to get the first and last name of the last member(s)
who signed up. Try not to use the LIMIT clause for your solution. */

SELECT `firstname`, `surname` FROM `Members` ORDER BY  `joindate` DESC LIMIT 10

/* Q7: Produce a list of all members who have used a tennis court.
Include in your output the name of the court, and the name of the member
formatted as a single column. Ensure no duplicate data, and order by
the member name. */

SELECT DISTINCT(concat(firstname,' ' , surname)), 
CASE WHEN (facid = 0) THEN "Tennis Court 1"
ELSE "Tennis Court 2"
END AS courtname
FROM Bookings
JOIN Members
ON Bookings.memid = Members.memid
WHERE facid = 0 OR facid = 1
ORDER BY surname

/* Q8: Produce a list of bookings on the day of 2012-09-14 which
will cost the member (or guest) more than $30. Remember that guests have
different costs to members (the listed costs are per half-hour 'slot'), and
the guest user's ID is always 0. Include in your output the name of the
facility, the name of the member formatted as a single column, and the cost.
Order by descending cost, and do not use any subqueries. */

SELECT DISTINCT (
CONCAT( firstname, ' ', surname )
) AS name, Facilities.name,
CASE WHEN (
Bookings.memid =0
)
THEN (
Bookings.slots * Facilities.guestcost
)
ELSE (
Bookings.slots * Facilities.membercost
)
END AS costs
FROM Bookings
INNER JOIN Members ON Bookings.memid = Members.memid
INNER JOIN Facilities ON Bookings.facid = Facilities.facid
WHERE starttime LIKE "2012-09-14%"
AND (
Members.memid =0
AND Bookings.slots * Facilities.guestcost >30
)
OR (
Members.memid !=0
AND Bookings.slots * Facilities.membercost >30
)
ORDER BY Bookings.memid






/* Q9: This time, produce the same result as in Q8, but using a subquery. */

SELECT member, facility, cost FROM (
SELECT 
concat(Members.firstname, ' ' , Members.surname) AS member,
Facilities.name as facility,
CASE
WHEN Members.memid = 0 THEN
Bookings.slots*Facilities.guestcost
ELSE
Bookings.slots*Facilities.membercost
END AS cost
FROM
Members
INNER JOIN Bookings
ON Members.memid = Bookings.memid
INNER JOIN Facilities
ON Bookings.facid = Facilities.facid
WHERE starttime LIKE "2012-09-14%" 
) AS bookings
WHERE cost > 30
ORDER BY cost DESC 


In [1]:
import sqlite3
from sqlite3 import Error

def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
 
    return conn
 
def select_all_tasks(conn):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    
    Q10 = """
        SELECT facility, total_revenue
        FROM (
        SELECT Facilities.name AS facility,
        SUM(
        CASE WHEN Bookings.memid !=0
        THEN Bookings.slots * Facilities.membercost
        ELSE Bookings.slots * Facilities.guestcost END) AS total_revenue
        FROM Facilities
        INNER JOIN Bookings ON Bookings.facid = Facilities.facid
        GROUP BY Facilities.name
        ) AS facility_total_revenue
        WHERE total_revenue < 1000
        ORDER BY total_revenue
        """
    
    
    cur.execute(Q10)
    
    rows = cur.fetchall()
 
    for row in rows:
        print(row)

In [2]:
database = "sqlite_db_pythonsqlite.db"
 
# create a database connection
conn = create_connection(database)
with conn: 
    print("10. Query all tasks")
    select_all_tasks(conn)

2.6.0
10. Query all tasks
('Table Tennis', 180)
('Snooker Table', 240)
('Pool Table', 270)


In [3]:
import sqlite3
from sqlite3 import Error

def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
 
    return conn
 
def select_all_tasks(conn):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    
    
    
    Q11 = """
        SELECT m.firstname,m.surname, m2.firstname,m2.surname
        FROM Members AS m
        INNER JOIN Members AS m2
        ON m.memid = m2.recommendedby
        WHERE m.recommendedby >=1
        ORDER BY m.surname

  
        """
   
   
    cur.execute(Q11)
    
    rows = cur.fetchall()
 
    for row in rows:
        print(row)

In [4]:
database = "sqlite_db_pythonsqlite.db"
 
# create a database connection
conn = create_connection(database)
with conn: 
    print("11. Query all tasks")
    select_all_tasks(conn)
    

2.6.0
11. Query all tasks
('Florence', 'Bader', 'Ramnaresh', 'Sarwin')
('Timothy', 'Baker', 'Joan', 'Coplin')
('Gerald', 'Butters', 'Matthew', 'Genting')
('Matthew', 'Genting', 'Henrietta', 'Rumney')
('David', 'Jones', 'Douglas', 'Jones')
('Janice', 'Joplette', 'Nancy', 'Dare')
('Janice', 'Joplette', 'David', 'Jones')
('Millicent', 'Purview', 'John', 'Hunt')
('Ponder', 'Stibbons', 'Anne', 'Baker')
('Ponder', 'Stibbons', 'Florence', 'Bader')


In [5]:
import sqlite3
from sqlite3 import Error

def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
 
    return conn
 
def select_all_tasks(conn):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    
    
      

  
        
    Q12 = """
        SELECT f.name, m.firstname, m.surname
        FROM Bookings AS b
        INNER JOIN Members AS m
        ON b.memid = m.memid
        INNER JOIN Facilities AS f
        ON b.facid = f.facid
        WHERE m.memid != 0 
        ORDER BY f.name
        """
   
   
    cur.execute(Q12)
    
    rows = cur.fetchall()
 
    for row in rows:
        print(row)

In [6]:
database = "sqlite_db_pythonsqlite.db"
 
# create a database connection
conn = create_connection(database)
with conn: 
    print("12. Query all tasks")
    select_all_tasks(conn)

2.6.0
12. Query all tasks
('Badminton Court', 'Darren', 'Smith')
('Badminton Court', 'Darren', 'Smith')
('Badminton Court', 'Darren', 'Smith')
('Badminton Court', 'Darren', 'Smith')
('Badminton Court', 'Darren', 'Smith')
('Badminton Court', 'Darren', 'Smith')
('Badminton Court', 'Darren', 'Smith')
('Badminton Court', 'Darren', 'Smith')
('Badminton Court', 'Darren', 'Smith')
('Badminton Court', 'Darren', 'Smith')
('Badminton Court', 'Darren', 'Smith')
('Badminton Court', 'Darren', 'Smith')
('Badminton Court', 'Darren', 'Smith')
('Badminton Court', 'Gerald', 'Butters')
('Badminton Court', 'Darren', 'Smith')
('Badminton Court', 'Darren', 'Smith')
('Badminton Court', 'Tracy', 'Smith')
('Badminton Court', 'Darren', 'Smith')
('Badminton Court', 'Darren', 'Smith')
('Badminton Court', 'Darren', 'Smith')
('Badminton Court', 'Tracy', 'Smith')
('Badminton Court', 'Darren', 'Smith')
('Badminton Court', 'Darren', 'Smith')
('Badminton Court', 'Darren', 'Smith')
('Badminton Court', 'Darren', 'Smith')

('Pool Table', 'Tim', 'Rownam')
('Pool Table', 'Tim', 'Rownam')
('Pool Table', 'Tracy', 'Smith')
('Pool Table', 'Tim', 'Rownam')
('Pool Table', 'Tim', 'Rownam')
('Pool Table', 'Tim', 'Rownam')
('Pool Table', 'Tim', 'Rownam')
('Pool Table', 'Janice', 'Joplette')
('Pool Table', 'Tim', 'Rownam')
('Pool Table', 'Tim', 'Rownam')
('Pool Table', 'Janice', 'Joplette')
('Pool Table', 'Darren', 'Smith')
('Pool Table', 'Tracy', 'Smith')
('Pool Table', 'Tim', 'Rownam')
('Pool Table', 'Tim', 'Rownam')
('Pool Table', 'Tracy', 'Smith')
('Pool Table', 'Darren', 'Smith')
('Pool Table', 'Tim', 'Rownam')
('Pool Table', 'Tim', 'Rownam')
('Pool Table', 'Tim', 'Rownam')
('Pool Table', 'Tim', 'Rownam')
('Pool Table', 'Burton', 'Tracy')
('Pool Table', 'Tracy', 'Smith')
('Pool Table', 'Tim', 'Rownam')
('Pool Table', 'Janice', 'Joplette')
('Pool Table', 'Tim', 'Rownam')
('Pool Table', 'Nancy', 'Dare')
('Pool Table', 'Tim', 'Rownam')
('Pool Table', 'Tim', 'Rownam')
('Pool Table', 'Tim', 'Rownam')
('Pool Table', 

('Snooker Table', 'Janice', 'Joplette')
('Snooker Table', 'Millicent', 'Purview')
('Snooker Table', 'Hyacinth', 'Tupperware')
('Snooker Table', 'David', 'Pinker')
('Snooker Table', 'Henrietta', 'Rumney')
('Snooker Table', 'Tim', 'Boothe')
('Snooker Table', 'David', 'Pinker')
('Snooker Table', 'Ponder', 'Stibbons')
('Snooker Table', 'Henrietta', 'Rumney')
('Snooker Table', 'Florence', 'Bader')
('Snooker Table', 'Jack', 'Smith')
('Snooker Table', 'Ramnaresh', 'Sarwin')
('Snooker Table', 'Joan', 'Coplin')
('Snooker Table', 'Ponder', 'Stibbons')
('Snooker Table', 'David', 'Pinker')
('Snooker Table', 'David', 'Farrell')
('Snooker Table', 'Charles', 'Owen')
('Snooker Table', 'Tim', 'Boothe')
('Snooker Table', 'Anna', 'Mackenzie')
('Snooker Table', 'Charles', 'Owen')
('Snooker Table', 'Nancy', 'Dare')
('Snooker Table', 'Hyacinth', 'Tupperware')
('Snooker Table', 'Tracy', 'Smith')
('Snooker Table', 'Nancy', 'Dare')
('Snooker Table', 'Ramnaresh', 'Sarwin')
('Snooker Table', 'Gerald', 'Butters')

('Tennis Court 2', 'Ponder', 'Stibbons')
('Tennis Court 2', 'Darren', 'Smith')
('Tennis Court 2', 'Tim', 'Boothe')
('Tennis Court 2', 'Tim', 'Rownam')
('Tennis Court 2', 'Ponder', 'Stibbons')
('Tennis Court 2', 'Ponder', 'Stibbons')
('Tennis Court 2', 'Tim', 'Boothe')
('Tennis Court 2', 'Nancy', 'Dare')
('Tennis Court 2', 'Charles', 'Owen')
('Tennis Court 2', 'Charles', 'Owen')
('Tennis Court 2', 'Ponder', 'Stibbons')
('Tennis Court 2', 'Charles', 'Owen')
('Tennis Court 2', 'Tim', 'Boothe')
('Tennis Court 2', 'Charles', 'Owen')
('Tennis Court 2', 'Tim', 'Boothe')
('Tennis Court 2', 'Ponder', 'Stibbons')
('Tennis Court 2', 'Charles', 'Owen')
('Tennis Court 2', 'David', 'Jones')
('Tennis Court 2', 'Tim', 'Boothe')
('Tennis Court 2', 'Ponder', 'Stibbons')
('Tennis Court 2', 'Tim', 'Boothe')
('Tennis Court 2', 'Anne', 'Baker')
('Tennis Court 2', 'Burton', 'Tracy')
('Tennis Court 2', 'Charles', 'Owen')
('Tennis Court 2', 'David', 'Jones')
('Tennis Court 2', 'David', 'Jones')
('Tennis Court 

In [7]:
import sqlite3
from sqlite3 import Error

def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
 
    return conn
 
def select_all_tasks(conn):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    
   
    Q13 = """
        SELECT Facilities.name, Bookings.facid, sum(slots) as "Total Slots"
        FROM Bookings
        INNER JOIN Facilities
        ON Bookings.facid = Facilities.facid
        WHERE starttime LIKE '%09-04%'
        GROUP BY Bookings.facid
        ORDER BY sum(slots);
        """
    
    cur.execute(Q13)
    rows = cur.fetchall()
 
    for row in rows:
        print(row)

In [8]:
database = "sqlite_db_pythonsqlite.db"
 
# create a database connection
conn = create_connection(database)
with conn: 
    print("13. Query all tasks")
    select_all_tasks(conn)

2.6.0
13. Query all tasks
('Massage Room 2', 5, 4)
('Table Tennis', 3, 12)
('Squash Court', 6, 12)
('Pool Table', 8, 15)
('Snooker Table', 7, 16)
('Tennis Court 1', 0, 18)
('Tennis Court 2', 1, 18)
('Massage Room 1', 4, 20)
('Badminton Court', 2, 21)
